# Discrimination between lateral and retroflex

In lab 1-2, we did the discrimination between lateral and retroflex with a simple logistic regression model. Here, we will investigate whether **deep and wide** learning helps the model to distinguish lateral and retroflex better than a simple logistic regression model. In lab 1-2, the accuracy from the simple logistic model was about 90%. 

## Check the data

In [1]:
%%bash
head data/liquid_regression.txt

Spk	Consonant	f2	f3
S1	retroflex	1441.277508	2869.284673
S1	retroflex	1196.771082	2455.18832
S1	retroflex	1171.343058	2062.017984
S1	retroflex	1328.246028	2539.038397
S1	retroflex	873.623052	2283.855398
S1	retroflex	1657.414313	3359.304476
S1	retroflex	1870.860324	2611.92996
S1	retroflex	1286.257271	2596.406174
S1	retroflex	821.9162893	1520.435742


## Define Min-Max normalization

In [6]:
def minmax_scale(original_array, target_array):
    (min_f2, min_f3) = np.min(original_array, axis = 0)
    (max_f2, max_f3) = np.max(original_array, axis = 0)
    for i in range(0, len(original_array)):
        for j in range(0, len(original_array[i])):
            if (j == 0):
                target_array[i][j] = (original_array[i][j] - min_f2) / (max_f2 - min_f2)
            else:
                target_array[i][j] = (original_array[i][j] - min_f3) / (max_f3 - min_f3)
    return target_array
            

## Import modules

In [7]:
import numpy as np
from keras import optimizers
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import TensorBoard

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load the data and create a dataset

In [4]:
with open("data/liquid_regression.txt", "r") as f:
    data = f.readlines()
    f.close()
    
import random

X_data = []
y_data = []

ndata = data[1:]
random.shuffle(ndata)

for i in ndata:

    line = i.strip()
    (spk, con, f2, f3) = line.split("\t")
    f2 = float(f2)
    f3 = float(f3)
    X = [f2, f3]

    if (con == "retroflex"):
        y = [0.]
    elif (con == "lateral"):
        y = [1.]
    X_data.append(X)
    y_data.append(y)

print(len(X_data))
print(X_data[0:20])
print(y_data[0:20])

360
[[838.8876552, 3168.771567], [929.774175, 2974.713792], [829.5601497, 2863.550953], [1059.918751, 2172.751751], [948.1321093, 1698.331262], [985.0487913, 2792.173045], [1345.301899, 2329.857775], [1095.29988, 2573.711217], [829.49253, 1422.719543], [1139.84887, 1286.326085], [995.9789782, 3027.117097], [859.8840934, 1396.976593], [1469.209023, 3480.820791], [1082.651276, 1512.312672], [747.6899461, 2489.599591], [1296.604478, 1730.181155], [1095.413051, 1834.390199], [994.4285363, 2842.843596], [1431.315319, 1558.991127], [755.8309011, 2669.579139]]
[[1.0], [1.0], [1.0], [0.0], [0.0], [1.0], [0.0], [1.0], [0.0], [0.0], [1.0], [0.0], [1.0], [0.0], [1.0], [0.0], [0.0], [1.0], [0.0], [1.0]]


## Do min-max normalization, and split the data into training, validation, and test dataset

In [8]:
X_zeros = np.zeros_like(X_data)
X_data = minmax_scale(X_data, X_zeros)

X_train = np.array(X_data[0:250])
y_train = np.array(y_data[0:250])

X_val = np.array(X_data[250:300])
y_val = np.array(y_data[250:300])

X_test = np.array(X_data[300:])
y_test = np.array(y_data[300:])

## Create a model with one hidden layer

In [9]:
model = Sequential()

model.add(Dense(2, input_dim = 2, activation = 'sigmoid'))
model.add(Dense(1, activation = 'sigmoid'))

adam = optimizers.Adam(lr=0.001)

model.compile(loss="binary_crossentropy", optimizer=adam, metrics = ['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


## Training the model

In [19]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq = 0, 
                         write_graph=True, write_images=False)

model.fit(X_train, y_train, batch_size = 10, epochs = 100, verbose = 1, validation_data=(X_val, y_val), 
          shuffle = True, 
          callbacks=[tensorboard])

Train on 250 samples, validate on 50 samples
Epoch 1/500
250/250 [==============================] - 0s 166us/step - loss: 0.5283 - acc: 0.9440 - val_loss: 0.5299 - val_acc: 0.9400
Epoch 2/500
250/250 [==============================] - 0s 207us/step - loss: 0.5256 - acc: 0.9440 - val_loss: 0.5275 - val_acc: 0.9400
Epoch 3/500
250/250 [==============================] - 0s 188us/step - loss: 0.5231 - acc: 0.9440 - val_loss: 0.5250 - val_acc: 0.9400
Epoch 4/500
250/250 [==============================] - 0s 193us/step - loss: 0.5205 - acc: 0.9440 - val_loss: 0.5226 - val_acc: 0.9400
Epoch 5/500
250/250 [==============================] - 0s 218us/step - loss: 0.5179 - acc: 0.9440 - val_loss: 0.5201 - val_acc: 0.9400
Epoch 6/500
250/250 [==============================] - 0s 187us/step - loss: 0.5154 - acc: 0.9440 - val_loss: 0.5177 - val_acc: 0.9400
Epoch 7/500
250/250 [==============================] - 0s 201us/step - loss: 0.5129 - acc: 0.9440 - val_loss: 0.5151 - val_acc: 0.9400
Epoch 8/50

Epoch 61/500
250/250 [==============================] - 0s 171us/step - loss: 0.3894 - acc: 0.9440 - val_loss: 0.3912 - val_acc: 0.9400
Epoch 62/500
250/250 [==============================] - 0s 167us/step - loss: 0.3875 - acc: 0.9440 - val_loss: 0.3894 - val_acc: 0.9400
Epoch 63/500
250/250 [==============================] - 0s 174us/step - loss: 0.3855 - acc: 0.9440 - val_loss: 0.3873 - val_acc: 0.9400
Epoch 64/500
250/250 [==============================] - 0s 197us/step - loss: 0.3836 - acc: 0.9440 - val_loss: 0.3853 - val_acc: 0.9400
Epoch 65/500
250/250 [==============================] - 0s 190us/step - loss: 0.3817 - acc: 0.9440 - val_loss: 0.3833 - val_acc: 0.9400
Epoch 66/500
250/250 [==============================] - 0s 160us/step - loss: 0.3799 - acc: 0.9440 - val_loss: 0.3814 - val_acc: 0.9400
Epoch 67/500
250/250 [==============================] - 0s 176us/step - loss: 0.3780 - acc: 0.9440 - val_loss: 0.3795 - val_acc: 0.9400
Epoch 68/500
250/250 [==========================

250/250 [==============================] - 0s 188us/step - loss: 0.2955 - acc: 0.9400 - val_loss: 0.2924 - val_acc: 0.9400
Epoch 122/500
250/250 [==============================] - 0s 195us/step - loss: 0.2943 - acc: 0.9400 - val_loss: 0.2909 - val_acc: 0.9400
Epoch 123/500
250/250 [==============================] - 0s 194us/step - loss: 0.2933 - acc: 0.9400 - val_loss: 0.2895 - val_acc: 0.9400
Epoch 124/500
250/250 [==============================] - 0s 188us/step - loss: 0.2919 - acc: 0.9400 - val_loss: 0.2884 - val_acc: 0.9400
Epoch 125/500
250/250 [==============================] - 0s 201us/step - loss: 0.2909 - acc: 0.9400 - val_loss: 0.2873 - val_acc: 0.9400
Epoch 126/500
250/250 [==============================] - 0s 188us/step - loss: 0.2897 - acc: 0.9400 - val_loss: 0.2860 - val_acc: 0.9400
Epoch 127/500
250/250 [==============================] - 0s 201us/step - loss: 0.2885 - acc: 0.9400 - val_loss: 0.2848 - val_acc: 0.9400
Epoch 128/500
250/250 [==============================] 

Epoch 181/500
250/250 [==============================] - 0s 181us/step - loss: 0.2418 - acc: 0.9400 - val_loss: 0.2310 - val_acc: 0.9400
Epoch 182/500
250/250 [==============================] - 0s 177us/step - loss: 0.2411 - acc: 0.9400 - val_loss: 0.2303 - val_acc: 0.9400
Epoch 183/500
250/250 [==============================] - 0s 169us/step - loss: 0.2404 - acc: 0.9400 - val_loss: 0.2292 - val_acc: 0.9400
Epoch 184/500
250/250 [==============================] - 0s 164us/step - loss: 0.2397 - acc: 0.9400 - val_loss: 0.2285 - val_acc: 0.9400
Epoch 185/500
250/250 [==============================] - 0s 174us/step - loss: 0.2391 - acc: 0.9400 - val_loss: 0.2277 - val_acc: 0.9400
Epoch 186/500
250/250 [==============================] - 0s 163us/step - loss: 0.2384 - acc: 0.9400 - val_loss: 0.2270 - val_acc: 0.9400
Epoch 187/500
250/250 [==============================] - 0s 163us/step - loss: 0.2380 - acc: 0.9400 - val_loss: 0.2260 - val_acc: 0.9400
Epoch 188/500
250/250 [==================

Epoch 241/500
250/250 [==============================] - 0s 169us/step - loss: 0.2120 - acc: 0.9400 - val_loss: 0.1933 - val_acc: 0.9400
Epoch 242/500
250/250 [==============================] - 0s 163us/step - loss: 0.2117 - acc: 0.9400 - val_loss: 0.1927 - val_acc: 0.9400
Epoch 243/500
250/250 [==============================] - 0s 181us/step - loss: 0.2114 - acc: 0.9400 - val_loss: 0.1924 - val_acc: 0.9400
Epoch 244/500
250/250 [==============================] - 0s 170us/step - loss: 0.2110 - acc: 0.9400 - val_loss: 0.1918 - val_acc: 0.9400
Epoch 245/500
250/250 [==============================] - 0s 173us/step - loss: 0.2109 - acc: 0.9400 - val_loss: 0.1916 - val_acc: 0.9400
Epoch 246/500
250/250 [==============================] - 0s 179us/step - loss: 0.2102 - acc: 0.9400 - val_loss: 0.1909 - val_acc: 0.9400
Epoch 247/500
250/250 [==============================] - 0s 185us/step - loss: 0.2099 - acc: 0.9400 - val_loss: 0.1905 - val_acc: 0.9400
Epoch 248/500
250/250 [==================

Epoch 301/500
250/250 [==============================] - 0s 185us/step - loss: 0.1960 - acc: 0.9360 - val_loss: 0.1698 - val_acc: 0.9400
Epoch 302/500
250/250 [==============================] - 0s 177us/step - loss: 0.1956 - acc: 0.9400 - val_loss: 0.1698 - val_acc: 0.9400
Epoch 303/500
250/250 [==============================] - 0s 164us/step - loss: 0.1954 - acc: 0.9400 - val_loss: 0.1695 - val_acc: 0.9400
Epoch 304/500
250/250 [==============================] - 0s 166us/step - loss: 0.1951 - acc: 0.9400 - val_loss: 0.1691 - val_acc: 0.9400
Epoch 305/500
250/250 [==============================] - 0s 164us/step - loss: 0.1950 - acc: 0.9400 - val_loss: 0.1688 - val_acc: 0.9400
Epoch 306/500
250/250 [==============================] - 0s 164us/step - loss: 0.1947 - acc: 0.9400 - val_loss: 0.1684 - val_acc: 0.9400
Epoch 307/500
250/250 [==============================] - 0s 161us/step - loss: 0.1946 - acc: 0.9400 - val_loss: 0.1683 - val_acc: 0.9400
Epoch 308/500
250/250 [==================

Epoch 361/500
250/250 [==============================] - 0s 177us/step - loss: 0.1863 - acc: 0.9400 - val_loss: 0.1547 - val_acc: 0.9400
Epoch 362/500
250/250 [==============================] - 0s 164us/step - loss: 0.1861 - acc: 0.9400 - val_loss: 0.1546 - val_acc: 0.9400
Epoch 363/500
250/250 [==============================] - 0s 164us/step - loss: 0.1861 - acc: 0.9400 - val_loss: 0.1543 - val_acc: 0.9400
Epoch 364/500
250/250 [==============================] - 0s 160us/step - loss: 0.1859 - acc: 0.9400 - val_loss: 0.1543 - val_acc: 0.9400
Epoch 365/500
250/250 [==============================] - 0s 174us/step - loss: 0.1858 - acc: 0.9400 - val_loss: 0.1540 - val_acc: 0.9400
Epoch 366/500
250/250 [==============================] - 0s 163us/step - loss: 0.1859 - acc: 0.9400 - val_loss: 0.1539 - val_acc: 0.9400
Epoch 367/500
250/250 [==============================] - 0s 158us/step - loss: 0.1858 - acc: 0.9400 - val_loss: 0.1536 - val_acc: 0.9400
Epoch 368/500
250/250 [==================

Epoch 421/500
250/250 [==============================] - 0s 173us/step - loss: 0.1806 - acc: 0.9400 - val_loss: 0.1447 - val_acc: 0.9400
Epoch 422/500
250/250 [==============================] - 0s 173us/step - loss: 0.1805 - acc: 0.9400 - val_loss: 0.1444 - val_acc: 0.9400
Epoch 423/500
250/250 [==============================] - 0s 160us/step - loss: 0.1807 - acc: 0.9400 - val_loss: 0.1445 - val_acc: 0.9400
Epoch 424/500
250/250 [==============================] - 0s 162us/step - loss: 0.1803 - acc: 0.9400 - val_loss: 0.1442 - val_acc: 0.9400
Epoch 425/500
250/250 [==============================] - 0s 163us/step - loss: 0.1803 - acc: 0.9400 - val_loss: 0.1440 - val_acc: 0.9400
Epoch 426/500
250/250 [==============================] - 0s 168us/step - loss: 0.1803 - acc: 0.9400 - val_loss: 0.1439 - val_acc: 0.9400
Epoch 427/500
250/250 [==============================] - 0s 172us/step - loss: 0.1801 - acc: 0.9400 - val_loss: 0.1437 - val_acc: 0.9400
Epoch 428/500
250/250 [==================

Epoch 481/500
250/250 [==============================] - 0s 169us/step - loss: 0.1771 - acc: 0.9400 - val_loss: 0.1377 - val_acc: 0.9400
Epoch 482/500
250/250 [==============================] - 0s 162us/step - loss: 0.1771 - acc: 0.9400 - val_loss: 0.1373 - val_acc: 0.9400
Epoch 483/500
250/250 [==============================] - 0s 175us/step - loss: 0.1769 - acc: 0.9400 - val_loss: 0.1373 - val_acc: 0.9400
Epoch 484/500
250/250 [==============================] - 0s 162us/step - loss: 0.1768 - acc: 0.9400 - val_loss: 0.1372 - val_acc: 0.9400
Epoch 485/500
250/250 [==============================] - 0s 167us/step - loss: 0.1768 - acc: 0.9400 - val_loss: 0.1372 - val_acc: 0.9400
Epoch 486/500
250/250 [==============================] - 0s 156us/step - loss: 0.1767 - acc: 0.9400 - val_loss: 0.1370 - val_acc: 0.9400
Epoch 487/500
250/250 [==============================] - 0s 171us/step - loss: 0.1767 - acc: 0.9400 - val_loss: 0.1369 - val_acc: 0.9400
Epoch 488/500
250/250 [==================

## Results

See the actual values and predictions

In [20]:
predictions = model.predict(X_test)
for j in range(0, 20):
    print(y_test[j], "\t", predictions[j][0], "\t", round(predictions[j][0]))

[0.] 	 0.013983783 	 0.0
[1.] 	 0.9130779 	 1.0
[1.] 	 0.8656264 	 1.0
[0.] 	 0.0045761988 	 0.0
[1.] 	 0.61414564 	 1.0
[1.] 	 0.81760854 	 1.0
[0.] 	 0.72538465 	 1.0
[0.] 	 0.10895866 	 0.0
[0.] 	 0.009825967 	 0.0
[1.] 	 0.98465466 	 1.0
[1.] 	 0.98975426 	 1.0
[0.] 	 0.0082448665 	 0.0
[0.] 	 0.02113763 	 0.0
[0.] 	 0.28880063 	 0.0
[0.] 	 0.2983932 	 0.0
[1.] 	 0.9810994 	 1.0
[0.] 	 0.016542265 	 0.0
[1.] 	 0.9391279 	 1.0
[0.] 	 0.0098706335 	 0.0
[0.] 	 0.3955756 	 0.0


Checek the performance

In [21]:
score = model.evaluate(X_test, y_test, verbose = 0)

print("Test score:\t", score[0])
print("Test accuracy:\t", score[1])

Test score:	 0.21710628469785054
Test accuracy:	 0.9166666547457377


Oops, the model is **overfitted**, where the model works very good on training data, but works worse on test data. However, the accuracy is still about 90%. 

We will add one more layer to see how the model will work. 

## Create model with two hidden layers (deeper)

In [28]:
model2 = Sequential()

model2.add(Dense(2, input_dim = 2, activation = 'sigmoid'))
model2.add(Dense(2, activation = 'sigmoid'))
model2.add(Dense(1, activation = 'sigmoid'))

adam = optimizers.Adam(lr=0.001)

model2.compile(loss="binary_crossentropy", optimizer=adam, metrics = ['accuracy'])

model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 3         
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [29]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq = 0, 
                         write_graph=True, write_images=False)

model2.fit(X_train, y_train, batch_size = 10, epochs = 100, verbose = 1, validation_data=(X_val, y_val), 
          shuffle = True, 
          callbacks=[tensorboard])

Train on 250 samples, validate on 50 samples
Epoch 1/100
250/250 [==============================] - 0s 943us/step - loss: 0.7136 - acc: 0.5160 - val_loss: 0.7281 - val_acc: 0.4800
Epoch 2/100
250/250 [==============================] - 0s 218us/step - loss: 0.7108 - acc: 0.5160 - val_loss: 0.7242 - val_acc: 0.4800
Epoch 3/100
250/250 [==============================] - 0s 253us/step - loss: 0.7081 - acc: 0.5160 - val_loss: 0.7210 - val_acc: 0.4800
Epoch 4/100
250/250 [==============================] - 0s 212us/step - loss: 0.7060 - acc: 0.5160 - val_loss: 0.7176 - val_acc: 0.4800
Epoch 5/100
250/250 [==============================] - 0s 211us/step - loss: 0.7038 - acc: 0.5160 - val_loss: 0.7157 - val_acc: 0.4800
Epoch 6/100
250/250 [==============================] - 0s 202us/step - loss: 0.7023 - acc: 0.5160 - val_loss: 0.7128 - val_acc: 0.4800
Epoch 7/100
250/250 [==============================] - 0s 227us/step - loss: 0.7007 - acc: 0.5160 - val_loss: 0.7107 - val_acc: 0.4800
Epoch 8/10

Epoch 61/100
250/250 [==============================] - 0s 203us/step - loss: 0.6903 - acc: 0.5160 - val_loss: 0.6932 - val_acc: 0.4800
Epoch 62/100
250/250 [==============================] - 0s 196us/step - loss: 0.6901 - acc: 0.5160 - val_loss: 0.6930 - val_acc: 0.4800
Epoch 63/100
250/250 [==============================] - 0s 202us/step - loss: 0.6900 - acc: 0.5160 - val_loss: 0.6929 - val_acc: 0.4800
Epoch 64/100
250/250 [==============================] - 0s 195us/step - loss: 0.6900 - acc: 0.5160 - val_loss: 0.6928 - val_acc: 0.4800
Epoch 65/100
250/250 [==============================] - 0s 206us/step - loss: 0.6899 - acc: 0.5160 - val_loss: 0.6928 - val_acc: 0.4800
Epoch 66/100
250/250 [==============================] - 0s 181us/step - loss: 0.6898 - acc: 0.5160 - val_loss: 0.6927 - val_acc: 0.4800
Epoch 67/100
250/250 [==============================] - 0s 207us/step - loss: 0.6897 - acc: 0.5160 - val_loss: 0.6927 - val_acc: 0.4800
Epoch 68/100
250/250 [==========================

It looks like the model works really bad. I will increase the number of epochs to see whether the accuracy increases. 

In [15]:
model2.fit(X_train, y_train, batch_size = 10, epochs = 500, verbose = 1, validation_data=(X_val, y_val), 
          shuffle = True, 
          callbacks=[tensorboard])

Train on 250 samples, validate on 50 samples
Epoch 1/500
250/250 [==============================] - 0s 188us/step - loss: 0.6779 - acc: 0.5160 - val_loss: 0.6802 - val_acc: 0.4800
Epoch 2/500
250/250 [==============================] - 0s 191us/step - loss: 0.6775 - acc: 0.5200 - val_loss: 0.6795 - val_acc: 0.5000
Epoch 3/500
250/250 [==============================] - 0s 187us/step - loss: 0.6772 - acc: 0.5240 - val_loss: 0.6793 - val_acc: 0.4800
Epoch 4/500
250/250 [==============================] - 0s 179us/step - loss: 0.6763 - acc: 0.5240 - val_loss: 0.6786 - val_acc: 0.5000
Epoch 5/500
250/250 [==============================] - 0s 189us/step - loss: 0.6757 - acc: 0.5240 - val_loss: 0.6780 - val_acc: 0.5000
Epoch 6/500
250/250 [==============================] - 0s 239us/step - loss: 0.6752 - acc: 0.5280 - val_loss: 0.6774 - val_acc: 0.5000
Epoch 7/500
250/250 [==============================] - 0s 212us/step - loss: 0.6748 - acc: 0.5640 - val_loss: 0.6767 - val_acc: 0.5000
Epoch 8/50

Epoch 61/500
250/250 [==============================] - 0s 179us/step - loss: 0.5892 - acc: 0.9400 - val_loss: 0.5905 - val_acc: 0.9400
Epoch 62/500
250/250 [==============================] - 0s 175us/step - loss: 0.5862 - acc: 0.9400 - val_loss: 0.5874 - val_acc: 0.9400
Epoch 63/500
250/250 [==============================] - 0s 165us/step - loss: 0.5832 - acc: 0.9400 - val_loss: 0.5844 - val_acc: 0.9400
Epoch 64/500
250/250 [==============================] - 0s 171us/step - loss: 0.5800 - acc: 0.9360 - val_loss: 0.5812 - val_acc: 0.9400
Epoch 65/500
250/250 [==============================] - 0s 176us/step - loss: 0.5769 - acc: 0.9360 - val_loss: 0.5777 - val_acc: 0.9400
Epoch 66/500
250/250 [==============================] - 0s 187us/step - loss: 0.5739 - acc: 0.9360 - val_loss: 0.5748 - val_acc: 0.9400
Epoch 67/500
250/250 [==============================] - 0s 202us/step - loss: 0.5705 - acc: 0.9400 - val_loss: 0.5716 - val_acc: 0.9400
Epoch 68/500
250/250 [==========================

250/250 [==============================] - 0s 173us/step - loss: 0.3768 - acc: 0.9480 - val_loss: 0.3734 - val_acc: 0.9400
Epoch 122/500
250/250 [==============================] - 0s 182us/step - loss: 0.3736 - acc: 0.9480 - val_loss: 0.3698 - val_acc: 0.9400
Epoch 123/500
250/250 [==============================] - 0s 188us/step - loss: 0.3707 - acc: 0.9440 - val_loss: 0.3665 - val_acc: 0.9400
Epoch 124/500
250/250 [==============================] - 0s 188us/step - loss: 0.3674 - acc: 0.9440 - val_loss: 0.3632 - val_acc: 0.9400
Epoch 125/500
250/250 [==============================] - 0s 168us/step - loss: 0.3644 - acc: 0.9440 - val_loss: 0.3603 - val_acc: 0.9400
Epoch 126/500
250/250 [==============================] - 0s 167us/step - loss: 0.3614 - acc: 0.9440 - val_loss: 0.3570 - val_acc: 0.9400
Epoch 127/500
250/250 [==============================] - 0s 190us/step - loss: 0.3585 - acc: 0.9480 - val_loss: 0.3541 - val_acc: 0.9400
Epoch 128/500
250/250 [==============================] 

Epoch 181/500
250/250 [==============================] - 0s 182us/step - loss: 0.2505 - acc: 0.9440 - val_loss: 0.2335 - val_acc: 0.9400
Epoch 182/500
250/250 [==============================] - 0s 167us/step - loss: 0.2493 - acc: 0.9480 - val_loss: 0.2325 - val_acc: 0.9400
Epoch 183/500
250/250 [==============================] - 0s 164us/step - loss: 0.2483 - acc: 0.9480 - val_loss: 0.2307 - val_acc: 0.9400
Epoch 184/500
250/250 [==============================] - 0s 189us/step - loss: 0.2470 - acc: 0.9480 - val_loss: 0.2297 - val_acc: 0.9400
Epoch 185/500
250/250 [==============================] - 0s 168us/step - loss: 0.2457 - acc: 0.9480 - val_loss: 0.2280 - val_acc: 0.9400
Epoch 186/500
250/250 [==============================] - 0s 184us/step - loss: 0.2446 - acc: 0.9480 - val_loss: 0.2266 - val_acc: 0.9400
Epoch 187/500
250/250 [==============================] - 0s 180us/step - loss: 0.2436 - acc: 0.9480 - val_loss: 0.2249 - val_acc: 0.9400
Epoch 188/500
250/250 [==================

Epoch 241/500
250/250 [==============================] - 0s 178us/step - loss: 0.2031 - acc: 0.9480 - val_loss: 0.1730 - val_acc: 0.9400
Epoch 242/500
250/250 [==============================] - 0s 192us/step - loss: 0.2028 - acc: 0.9480 - val_loss: 0.1727 - val_acc: 0.9400
Epoch 243/500
250/250 [==============================] - 0s 162us/step - loss: 0.2022 - acc: 0.9480 - val_loss: 0.1720 - val_acc: 0.9400
Epoch 244/500
250/250 [==============================] - 0s 167us/step - loss: 0.2019 - acc: 0.9480 - val_loss: 0.1711 - val_acc: 0.9400
Epoch 245/500
250/250 [==============================] - 0s 167us/step - loss: 0.2014 - acc: 0.9480 - val_loss: 0.1703 - val_acc: 0.9400
Epoch 246/500
250/250 [==============================] - 0s 163us/step - loss: 0.2009 - acc: 0.9480 - val_loss: 0.1697 - val_acc: 0.9400
Epoch 247/500
250/250 [==============================] - 0s 171us/step - loss: 0.2006 - acc: 0.9440 - val_loss: 0.1691 - val_acc: 0.9400
Epoch 248/500
250/250 [==================

Epoch 301/500
250/250 [==============================] - 0s 174us/step - loss: 0.1850 - acc: 0.9480 - val_loss: 0.1452 - val_acc: 0.9400
Epoch 302/500
250/250 [==============================] - 0s 166us/step - loss: 0.1849 - acc: 0.9480 - val_loss: 0.1450 - val_acc: 0.9400
Epoch 303/500
250/250 [==============================] - 0s 180us/step - loss: 0.1846 - acc: 0.9480 - val_loss: 0.1445 - val_acc: 0.9400
Epoch 304/500
250/250 [==============================] - 0s 167us/step - loss: 0.1845 - acc: 0.9480 - val_loss: 0.1439 - val_acc: 0.9400
Epoch 305/500
250/250 [==============================] - 0s 166us/step - loss: 0.1844 - acc: 0.9480 - val_loss: 0.1437 - val_acc: 0.9400
Epoch 306/500
250/250 [==============================] - 0s 180us/step - loss: 0.1844 - acc: 0.9480 - val_loss: 0.1441 - val_acc: 0.9400
Epoch 307/500
250/250 [==============================] - 0s 185us/step - loss: 0.1840 - acc: 0.9480 - val_loss: 0.1433 - val_acc: 0.9400
Epoch 308/500
250/250 [==================

Epoch 361/500
250/250 [==============================] - 0s 186us/step - loss: 0.1777 - acc: 0.9440 - val_loss: 0.1305 - val_acc: 0.9400
Epoch 362/500
250/250 [==============================] - 0s 187us/step - loss: 0.1773 - acc: 0.9480 - val_loss: 0.1311 - val_acc: 0.9400
Epoch 363/500
250/250 [==============================] - 0s 191us/step - loss: 0.1772 - acc: 0.9480 - val_loss: 0.1311 - val_acc: 0.9400
Epoch 364/500
250/250 [==============================] - 0s 186us/step - loss: 0.1770 - acc: 0.9480 - val_loss: 0.1309 - val_acc: 0.9400
Epoch 365/500
250/250 [==============================] - 0s 193us/step - loss: 0.1771 - acc: 0.9480 - val_loss: 0.1306 - val_acc: 0.9400
Epoch 366/500
250/250 [==============================] - 0s 177us/step - loss: 0.1769 - acc: 0.9480 - val_loss: 0.1300 - val_acc: 0.9400
Epoch 367/500
250/250 [==============================] - 0s 189us/step - loss: 0.1769 - acc: 0.9480 - val_loss: 0.1299 - val_acc: 0.9400
Epoch 368/500
250/250 [==================

Epoch 421/500
250/250 [==============================] - 0s 197us/step - loss: 0.1745 - acc: 0.9440 - val_loss: 0.1224 - val_acc: 0.9400
Epoch 422/500
250/250 [==============================] - 0s 180us/step - loss: 0.1736 - acc: 0.9480 - val_loss: 0.1226 - val_acc: 0.9400
Epoch 423/500
250/250 [==============================] - 0s 175us/step - loss: 0.1735 - acc: 0.9480 - val_loss: 0.1233 - val_acc: 0.9400
Epoch 424/500
250/250 [==============================] - 0s 165us/step - loss: 0.1734 - acc: 0.9480 - val_loss: 0.1233 - val_acc: 0.9400
Epoch 425/500
250/250 [==============================] - 0s 184us/step - loss: 0.1734 - acc: 0.9480 - val_loss: 0.1232 - val_acc: 0.9400
Epoch 426/500
250/250 [==============================] - 0s 199us/step - loss: 0.1735 - acc: 0.9480 - val_loss: 0.1226 - val_acc: 0.9400
Epoch 427/500
250/250 [==============================] - 0s 200us/step - loss: 0.1733 - acc: 0.9480 - val_loss: 0.1226 - val_acc: 0.9400
Epoch 428/500
250/250 [==================

Epoch 481/500
250/250 [==============================] - 0s 184us/step - loss: 0.1714 - acc: 0.9480 - val_loss: 0.1184 - val_acc: 0.9400
Epoch 482/500
250/250 [==============================] - 0s 175us/step - loss: 0.1714 - acc: 0.9480 - val_loss: 0.1185 - val_acc: 0.9400
Epoch 483/500
250/250 [==============================] - 0s 165us/step - loss: 0.1713 - acc: 0.9480 - val_loss: 0.1188 - val_acc: 0.9400
Epoch 484/500
250/250 [==============================] - 0s 171us/step - loss: 0.1715 - acc: 0.9480 - val_loss: 0.1184 - val_acc: 0.9400
Epoch 485/500
250/250 [==============================] - 0s 182us/step - loss: 0.1714 - acc: 0.9480 - val_loss: 0.1187 - val_acc: 0.9400
Epoch 486/500
250/250 [==============================] - 0s 167us/step - loss: 0.1716 - acc: 0.9480 - val_loss: 0.1177 - val_acc: 0.9400
Epoch 487/500
250/250 [==============================] - 0s 165us/step - loss: 0.1714 - acc: 0.9480 - val_loss: 0.1188 - val_acc: 0.9400
Epoch 488/500
250/250 [==================

## Results

See the actual values and predicted values

In [16]:
predictions = model2.predict(X_test)
for j in range(0, 20):
    print(y_test[j], "\t", predictions[j][0], "\t", round(predictions[j][0]))

[0.] 	 0.013323543 	 0.0
[1.] 	 0.94754195 	 1.0
[1.] 	 0.9236921 	 1.0
[0.] 	 0.008191694 	 0.0
[1.] 	 0.6792656 	 1.0
[1.] 	 0.8565961 	 1.0
[0.] 	 0.7787437 	 1.0
[0.] 	 0.06652434 	 0.0
[0.] 	 0.011170023 	 0.0
[1.] 	 0.9781214 	 1.0
[1.] 	 0.97995996 	 1.0
[0.] 	 0.010315532 	 0.0
[0.] 	 0.017112292 	 0.0
[0.] 	 0.23729339 	 0.0
[0.] 	 0.27088723 	 0.0
[1.] 	 0.9769355 	 1.0
[0.] 	 0.01482139 	 0.0
[1.] 	 0.9601932 	 1.0
[0.] 	 0.011185133 	 0.0
[0.] 	 0.4071008 	 0.0


Check the performance of the model

In [30]:
score = model2.evaluate(X_test, y_test, verbose = 0)

print("Test score:\t", score[0])
print("Test accuracy:\t", score[1])

Test score:	 0.6899616320927938
Test accuracy:	 0.45


Good. Now we have about 92% accuracy, which means that adding more layer actually helps the model. 

----------

*FYI) When you increase the number of epochs in the model with one hidden layer, you will have the same accuracy with this model. So, we are still not sure which one was more effective, the number of epochs or the number of hidden layers. We will take a look at it if we have a time.*

----------


Now, we will investigate whether the wider model (adding more nodes) will help the model to have better performance. 

## Create a model with one hidden layer, with more nodes

In [22]:
model3 = Sequential()

model3.add(Dense(10, input_dim = 2, activation = 'sigmoid'))
model3.add(Dense(1, activation = 'sigmoid'))

adam = optimizers.Adam(lr=0.001)

model3.compile(loss="binary_crossentropy", optimizer=adam, metrics = ['accuracy'])

model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 10)                30        
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 11        
Total params: 41
Trainable params: 41
Non-trainable params: 0
_________________________________________________________________


Now, train the model

In [23]:
model3.fit(X_train, y_train, batch_size = 10, epochs = 100, verbose = 1, validation_data=(X_val, y_val), 
          shuffle = True, 
          callbacks=[tensorboard])

Train on 250 samples, validate on 50 samples
Epoch 1/100
250/250 [==============================] - 0s 710us/step - loss: 0.6994 - acc: 0.5160 - val_loss: 0.7061 - val_acc: 0.4800
Epoch 2/100
250/250 [==============================] - 0s 200us/step - loss: 0.6934 - acc: 0.5160 - val_loss: 0.6979 - val_acc: 0.4800
Epoch 3/100
250/250 [==============================] - 0s 209us/step - loss: 0.6884 - acc: 0.5160 - val_loss: 0.6933 - val_acc: 0.4800
Epoch 4/100
250/250 [==============================] - 0s 184us/step - loss: 0.6851 - acc: 0.5160 - val_loss: 0.6888 - val_acc: 0.4800
Epoch 5/100
250/250 [==============================] - 0s 204us/step - loss: 0.6821 - acc: 0.5160 - val_loss: 0.6859 - val_acc: 0.4800
Epoch 6/100
250/250 [==============================] - 0s 187us/step - loss: 0.6794 - acc: 0.5160 - val_loss: 0.6822 - val_acc: 0.4800
Epoch 7/100
250/250 [==============================] - 0s 183us/step - loss: 0.6771 - acc: 0.5160 - val_loss: 0.6789 - val_acc: 0.4800
Epoch 8/10

Epoch 61/100
250/250 [==============================] - 0s 200us/step - loss: 0.5073 - acc: 0.9320 - val_loss: 0.5116 - val_acc: 0.9200
Epoch 62/100
250/250 [==============================] - 0s 177us/step - loss: 0.5031 - acc: 0.9320 - val_loss: 0.5073 - val_acc: 0.9200
Epoch 63/100
250/250 [==============================] - 0s 167us/step - loss: 0.4991 - acc: 0.9320 - val_loss: 0.5028 - val_acc: 0.9200
Epoch 64/100
250/250 [==============================] - 0s 167us/step - loss: 0.4949 - acc: 0.9320 - val_loss: 0.4987 - val_acc: 0.9200
Epoch 65/100
250/250 [==============================] - 0s 222us/step - loss: 0.4908 - acc: 0.9320 - val_loss: 0.4949 - val_acc: 0.9200
Epoch 66/100
250/250 [==============================] - 0s 258us/step - loss: 0.4866 - acc: 0.9320 - val_loss: 0.4908 - val_acc: 0.9200
Epoch 67/100
250/250 [==============================] - 0s 228us/step - loss: 0.4825 - acc: 0.9320 - val_loss: 0.4868 - val_acc: 0.9200
Epoch 68/100
250/250 [==========================

## Results

See the actual values and predicted values


In [25]:
predictions = model3.predict(X_test)
for j in range(0, 20):
    print(y_test[j], "\t", predictions[j][0], "\t", round(predictions[j][0]))

[0.] 	 0.21776406 	 0.0
[1.] 	 0.6759973 	 1.0
[1.] 	 0.63309723 	 1.0
[0.] 	 0.14060783 	 0.0
[1.] 	 0.53833777 	 1.0
[1.] 	 0.65783983 	 1.0
[0.] 	 0.5942651 	 1.0
[0.] 	 0.3679001 	 0.0
[0.] 	 0.18991858 	 0.0
[1.] 	 0.83425605 	 1.0
[1.] 	 0.8763221 	 1.0
[0.] 	 0.17455065 	 0.0
[0.] 	 0.24028806 	 0.0
[0.] 	 0.44520822 	 0.0
[0.] 	 0.4374966 	 0.0
[1.] 	 0.8084769 	 1.0
[0.] 	 0.21993099 	 0.0
[1.] 	 0.70446265 	 1.0
[0.] 	 0.1930156 	 0.0
[0.] 	 0.46592373 	 0.0


Check the performance

In [26]:
score = model3.evaluate(X_test, y_test, verbose = 0)

print("Test score:\t", score[0])
print("Test accuracy:\t", score[1])

Test score:	 0.3697415073712667
Test accuracy:	 0.9166666547457377


We can see the results are **the same**, but the model 3 (the model with one hidden layer, and more nodes) showed 92% accuracy with 100 epochs, wheras model 2 showed 45% accuracy with 100 epochs and model 1 showed about 90% accuracy with 100 epochs. So, with 100 epochs, model 3 showed the best accuracy on the given test data.

# PLAY WITH THE MODEL

Now, you know how to **add hidden layers**, and how to modify **the number of nodes in the hidden layer**. Change the structure of the model, and build your own. Check the performance of your model, and find a way to increase the performance of your model. If you happen to know how to use different **activation functions**, you can use it. 